# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.


### Part II - VacationPy

Now let's use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

* **Note:** Remember that any API usage beyond the $200 credit will be charged to your personal account. You can set quotas and limits to your daily requests to be sure you can't be charged. Check out [Google Maps Platform Billing](https://developers.google.com/maps/billing/gmp-billing#monitor-and-restrict-consumption) and [Manage your cost of use](https://developers.google.com/maps/documentation/javascript/usage-and-billing#set-caps) for more information.

* **Note:** if you having trouble displaying the maps try running `jupyter nbextension enable --py gmaps` in your environment and retry.

* Create a heat map that displays the humidity for every city from the part I of the homework.

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import ipywidgets as widgets
from ipywidgets.embed import embed_minimal_html

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = os.path.join('..', 'WeatherPy', 'Data', 'City_Weather.csv')
city_df = pd.read_csv(csv_file)
len(city_df)

555

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
max_intensity =  humidity.max()
max_intensity

100

In [4]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
max_intensity =  humidity.max()
#Use the Lat and Lng as locations and Humidity as the weight. Add Heatmap layer to map.
fig = gmaps.figure(layout={
        'width': '800px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,max_intensity=100, dissipating=True)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))

In [5]:
embed_minimal_html("Data/Humidity_Heatmap.html", views=[fig]);

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Narrow down the cities to fit weather conditions.
# A max temperature lower than 80 degrees F but higher than 70 F.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

pref_city_df = city_df.loc[(city_df["Wind Speed"] < 10) & (city_df["Max Temp"] > 70) & (city_df["Max Temp"] < 80) & (city_df["Cloudiness"] == 0), :]

#Drop any rows will null values.
pref_city_df = pref_city_df.dropna(how='any')
pref_city_df.reset_index(inplace=True)
del pref_city_df['index']
pref_city_df 

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,170,Sabinas,27.8500,-101.1167,75.47,73,0,2.39,MX,1631797636
1,201,Semey,50.4111,80.2275,71.82,21,0,8.95,KZ,1631797931
2,213,Yining,29.0361,114.5636,73.56,77,0,2.66,CN,1631797710
3,389,Atasū,48.6814,71.6433,72.77,19,0,8.32,KZ,1631798138
4,392,Vila,42.0304,-8.1588,70.70,62,0,3.80,PT,1631798141
5,409,Lata,40.1629,-8.3327,78.35,50,0,4.41,PT,1631798159
6,475,Blankenberge,51.3131,3.1323,72.25,64,0,1.99,BE,1631798234
7,507,Qingdao,36.0986,120.3719,73.00,83,0,6.71,CN,1631798225


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.

hotel_df=pref_city_df
pref_city_df['Hotel Name'] = ""
pref_city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,170,Sabinas,27.8500,-101.1167,75.47,73,0,2.39,MX,1631797636,
1,201,Semey,50.4111,80.2275,71.82,21,0,8.95,KZ,1631797931,
2,213,Yining,29.0361,114.5636,73.56,77,0,2.66,CN,1631797710,
3,389,Atasū,48.6814,71.6433,72.77,19,0,8.32,KZ,1631798138,
4,392,Vila,42.0304,-8.1588,70.70,62,0,3.80,PT,1631798141,
5,409,Lata,40.1629,-8.3327,78.35,50,0,4.41,PT,1631798159,
6,475,Blankenberge,51.3131,3.1323,72.25,64,0,1.99,BE,1631798234,
7,507,Qingdao,36.0986,120.3719,73.00,83,0,6.71,CN,1631798225,


In [8]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.

for index, row in hotel_df.iterrows():
    try:      
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }
        lat = row['Lat']
        lng = row['Lng']
        params['location'] = f"{lat}, {lng}"
        hotel_data = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
#pprint(hotel_data,indent=2)
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,170,Sabinas,27.8500,-101.1167,75.47,73,0,2.39,MX,1631797636,Hotel Posada Santa Fe
1,201,Semey,50.4111,80.2275,71.82,21,0,8.95,KZ,1631797931,Nomad Hotel
2,213,Yining,29.0361,114.5636,73.56,77,0,2.66,CN,1631797710,Xiuhe Hot Spring Holiday Hotel
3,389,Atasū,48.6814,71.6433,72.77,19,0,8.32,KZ,1631798138,NaN
4,392,Vila,42.0304,-8.1588,70.70,62,0,3.80,PT,1631798141,Hotel Castrum Villae
5,409,Lata,40.1629,-8.3327,78.35,50,0,4.41,PT,1631798159,NaN
6,475,Blankenberge,51.3131,3.1323,72.25,64,0,1.99,BE,1631798234,Mercure Blankenberge Station
7,507,Qingdao,36.0986,120.3719,73.00,83,0,6.71,CN,1631798225,Shangri-La Hotel Qingdao


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig



Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))

In [11]:
#save widget in html

embed_minimal_html("Data/Hotel_Map.html", views=[fig]);